# JUWELS at JSC compute node characteristics

Free memory for `batch`, `devel`, `esm` and `large` partitions,

In [1]:
%%bash
srun --account=esmtst --time=00:01:00 --partition=batch top -b -n 1 | head -n 5

top - 17:12:32 up  1:40,  1 user,  load average: 14.19, 36.95, 34.24
Tasks: 929 total,   1 running, 928 sleeping,   0 stopped,   0 zombie
%Cpu(s):  0.2 us,  0.4 sy,  0.0 ni, 99.4 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
MiB Mem :  94806.0 total,  76286.3 free,   5530.6 used,  12989.2 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.  85798.4 avail Mem 


In [2]:
!scontrol -o show nodes > node_state.txt

In [3]:
with open('node_state.txt') as f:
    lines = f.readlines()

In [4]:
matchers = ['NodeName=', 'CPUTot=', 'RealMemory=', 'FreeMem=', 'State=', 'Partitions=']
with open('node_state_parsed.txt','w') as f:
    for i in range(len(lines)):
        my_list = lines[i].split(' ')
        matching = [s for s in my_list if any(xs in s for xs in matchers)]
        f.write(';'.join(matching)+'\n')

In [5]:
import pandas

In [6]:
c1 = lambda x: (x.replace('NodeName=',''))
c2 = lambda x: (x.replace('RealMemory=',''))
c3 = lambda x: (x.replace('FreeMem=',''))
c4 = lambda x: (x.replace('State=',''))
c5 = lambda x: (x.replace('Partitions=',''))
c6 = lambda x: (x.replace('CPUTot=',''))

In [7]:
df = pandas.read_csv('node_state_parsed.txt', delimiter=';', header=None,
    names=['NodeName', 'CPUTot', 'RealMemory', 'FreeMem', 'State', 'Partitions'],
    converters={'NodeName':c1, 'RealMemory':c2, 'FreeMem':c3, 'State':c4, 'Partitions':c5, 'CPUTot':c6},
)

In [8]:
df

,NodeName,CPUTot,RealMemory,FreeMem,State,Partitions
0,jwb0001,96,515712,487030,ALLOCATED,"develbooster,maintbooster,maint"
1,jwb0002,96,515712,487032,RESERVED,"booster,largebooster,maintbooster,maint"
2,jwb0003,96,515712,486806,RESERVED,"booster,largebooster,maintbooster,maint"
3,jwb0004,96,515712,485974,RESERVED,"booster,largebooster,maintbooster,maint"
4,jwb0005,96,515712,486637,RESERVED,"booster,largebooster,maintbooster,maint"
...,...,...,...,...,...,...
3498,jwc09n165,80,180000,N/A,DOWN*+DRAIN,"gpus,maintcluster,maint"
3499,jwc09n174,80,180000,157559,ALLOCATED,"gpus,maintcluster,maint"
3500,jwc09n177,80,180000,146424,ALLOCATED,"gpus,maintcluster,maint"
3501,jwc09n180,80,180000,133886,ALLOCATED,"gpus,maintcluster,maint"


In [9]:
mask = df['Partitions'].str.contains('|'.join(['batch', 'devel', 'esm', 'large'])) \
& df['State'].str.contains('IDLE') \
& ~df['Partitions'].str.contains('mem192')

In [10]:
print(df[mask]['RealMemory'].max(), df[mask]['FreeMem'].max()) # MiB

90000 86866


Mount points for `batch` partition compute nodes,

In [11]:
%%bash
srun --account=esmtst --time=00:01:00 --partition=batch df

Filesystem           1K-blocks          Used     Available Use% Mounted on
devtmpfs              48512736             0      48512736   0% /dev
tmpfs                 48540680          2044      48538636   1% /run
tmpfs                 48540680             0      48540680   0% /sys/fs/cgroup
tmpfs                 48540680       2763732      45776948   6% /
tmpfs                 48540680             4      48540676   1% /dev/shm
tmpfs                  9708136             0       9708136   0% /run/user/0
scratch         14850371616768 6745390759936 8104980856832  46% /p/scratch
fastdata         9900247744512 7748817567744 2151430176768  79% /p/fastdata
hpcmon             17423376384    9409325056    8014051328  55% /p/hpcmon
project          4125103226880 2391816388608 1733286838272  58% /p/project
usersoftware       34458402816     359219200   34099183616   2% /p/usersoftware
juwels_software    17229201408    4362436608   12866764800  26% /p/software/juwels
home               64509345792

srun: job 3256254 queued and waiting for resources
srun: job 3256254 has been allocated resources


Python environment,

In [12]:
!conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://repo.anaconda.com/pkgs/main/linux-64/_libgcc_mutex-0.1-main.conda
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2020.12.5-ha878542_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/ld_impl_linux-64-2.33.1-h53a641e_7.conda
https://conda.anaconda.org/conda-forge/linux-64/libgfortran4-7.5.0-hae1eefd_17.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libstdcxx-ng-9.1.0-hdf63c60_0.conda
https://conda.anaconda.org/conda-forge/linux-64/pandoc-2.11.3.2-h7f98852_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgcc-ng-9.1.0-hdf63c60_0.conda
https://conda.anaconda.org/conda-forge/linux-64/libgfortran-ng-7.5.0-hae1eefd_17.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/jpeg-9d-h36c2ea0_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libffi-3.3-he6710b0_2.conda
https://conda.anaconda.org/conda-forge/linux-